# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 52 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.03264


extracting tarball to tmp_2307.03264...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


...

/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure rm_ropt
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure rm_ropt as tmp_2307.03264/./rm_ropt.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure rm_rnir
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure rm_rnir as tmp_2307.03264/./rm_rnir.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 65 bibliographic references in tmp_2307.03264/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2307.03369


extracting tarball to tmp_2307.03369...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.03264-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.03264) | **Stellar Half-Mass Radii of $0.5<z<2.3$ Galaxies: Comparison with  JWST/NIRCam Half-Light Radii**  |
|| A. v. d. Wel, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-07-10*|
|*Comments*| *Submitted to ApJ. Comments welcome*|
|**Abstract**| We use CEERS JWST/NIRCam imaging to measure rest-frame near-IR light profiles of $>$500 $M_\star>10^{10}~M_\odot$ galaxies in the redshift range $0.5<z<2.3$. We compare the resulting rest-frame 1.5-2$\mu$m half-light radii ($R_{\rm{NIR}}$) with stellar half-mass radii (\rmass) derived with multi-color light profiles from CANDELS HST imaging. In general agreement with previous work, we find that $R_{\rm{NIR}}$ and \rmass~are up to 40\%~smaller than the rest-frame optical half-light radius $R_{\rm{opt}}$. The agreement between $R_{\rm{NIR}}$ and \rmass~is excellent, with negligible systematic offset ($<$0.03 dex) up to $z=2$ for quiescent galaxies and up to $z=1.5$ for star-forming galaxies. We also deproject the profiles to estimate \rmassd, the radius of a sphere containing 50\% of the stellar mass. We present the $R-M_\star$ distribution of galaxies at $0.5<z<1.5$, comparing $R_{\rm{opt}}$, \rmass~and \rmassd. The slope is significantly flatter for \rmass~and \rmassd~ compared to $R_{\rm{opt}}$, mostly due to downward shifts in size for massive star-forming galaxies, while \rmass~and \rmassd~do not show markedly different trends. Finally, we show rapid size evolution ($R\propto (1+z)^{-1.7\pm0.1}$) for massive ($M_\star>10^{11}~M_\odot$) quiescent galaxies between $z=0.5$ and $z=2.3$, again comparing $R_{\rm{opt}}$, \rmass~and \rmassd. We conclude that the main tenets of the size evolution narrative established over the past 20 years, based on rest-frame optical light profile analysis, still hold in the era of JWST/NIRCam observations in the rest-frame near-IR. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.03369-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.03369) | **Molecular environment of the thermal composite supernova remnant  G352.7$-$0.1**  |
|| Q.-Q. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-10*|
|*Comments*| *14 pages, 10 figures, accepted for publication in ApJ*|
|**Abstract**| Galactic supernova remnants (SNRs) play an important role in our understanding of supernovae and their feedback on the interstellar environment. SNR G352.7$-$0.1 is special for its thermal composite morphology and double-ring structure. We have performed spectroscopic mapping in $^{12}$CO and $^{13}$CO $J=2$-1 lines toward G352.7$-$0.1 with the Atacama Pathfinder Experiment telescope. Broad $^{12}$CO lines are found in the northeastern ring at a local-standard-of-rest velocity range of $\sim-50$-$-30$ km s$^{-1}$, suggesting that the remnant is interacting with molecular clouds (MCs) at $\sim-51$ km s$^{-1}$. Thus, we adopt a distance of $\sim10.5$ kpc for this SNR. The momentum and kinetic energy of the shocked gas along the line of sight are estimated to be $\sim10^2{\rm M_{sun}}$ km s$^{-1}$ and $\sim10^{46}$ erg, respectively. We also find an expanding structure around the remnant, which is possibly related to the wind-blown bubble of the progenitor star. From the Fermi-LAT data in an energy range of 0.1-500 GeV, we find no gamma-ray counterparts of G352.7$-$0.1. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.03264.md
    + _build/html/tmp_2307.03264/./rm_ropt.png
    + _build/html/tmp_2307.03264/./m_r1.png
    + _build/html/tmp_2307.03264/./m_r2.png
    + _build/html/tmp_2307.03264/./ml_ml.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sigpr}{\sigma'_{\star,\rm{int}}}$
$\newcommand{\sigprsq}{\sigma'^2_{\star,\rm{int}}}$
$\newcommand{\ropt}{R_{0.5\mu\rm{m}}}$
$\newcommand{\rnirtwo}{R_{2.0\mu m}}$
$\newcommand{\rmass}{R_{\rm{M_\star}}}$
$\newcommand{\rmassd}{R_{\rm{M_\star,3D}}}$
$\newcommand{\vsig}{v_{5}/\sigma_{0}}$
$\newcommand{\rnirone}{R_{1.5\mu m}}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>



<div id="title">

# Stellar Half-Mass Radii of $0.5<z<2.3$ Galaxies: Comparison with JWST/NIRCam Half-Light Radii

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.03264-b31b1b.svg)](https://arxiv.org/abs/2307.03264)<mark>Appeared on: 2023-07-10</mark> -  _Submitted to ApJ. Comments welcome_

</div>
<div id="authors">

A. v. d. Wel, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We use CEERS JWST/NIRCam imaging to measure rest-frame near-IR light profiles of $>$ 500 $M_\star>10^{10} M_\odot$ galaxies in the redshift range $0.5<z<2.3$ . We compare the resulting rest-frame 1.5-2 $\mu$ m half-light radii ( $R_{\rm{NIR}}$ ) with stellar half-mass radii ( $\rmass$ ) derived with multi-color light profiles from CANDELS HST imaging. In general agreement with previous work, we find that $R_{\rm{NIR}}$ and $\rmass$ are  up to 40 \% smaller than the rest-frame optical half-light radius $R_{\rm{opt}}$ .The agreement between $R_{\rm{NIR}}$ and $\rmass$ is excellent, with negligible systematic offset ( $<$ 0.03 dex) up to $z=2$ for quiescent galaxies and up to $z=1.5$ for star-forming galaxies. We also deproject the profiles to estimate $\rmassd$ , the radius of a sphere containing 50 \% of the stellar mass.We present the $R-M_\star$ distribution of galaxies at $0.5<z<1.5$ , comparing $R_{\rm{opt}}$ , $\rmass$ and $\rmassd$ .The slope is significantly flatter for $\rmass$ and $\rmassd$ compared to $R_{\rm{opt}}$ , mostly due to downward shifts in size for massive star-forming galaxies, while $\rmass$ and $\rmassd$ do not show markedly different trends.Finally, we show rapid size evolution ( $R\propto (1+z)^{-1.7\pm0.1}$ ) for massive ( $M_\star>10^{11} M_\odot$ ) quiescent galaxies between $z=0.5$ and $z=2.3$ , again comparing $R_{\rm{opt}}$ , $\rmass$ and $\rmassd$ .We conclude that the main tenets of the size evolution narrative established over the past 20 years, based on rest-frame optical light profile analysis, still hold in the era of JWST/NIRCam observations in the rest-frame near-IR.

</div>

<div id="div_fig1">

<img src="tmp_2307.03264/./rm_ropt.png" alt="Fig7" width="100%"/>

**Figure 7. -**  Stellar half-mass radius vs. rest-frame optical half-light radius in four redshift bins for quiescent (red) and star-forming (blue) galaxies with total stellar mass $>10^{10} M_\odot$. $\Delta$ is the median offset in dex, with the 16-84\%-ile scatter in parentheses. $\sigma$ is the median formal uncertainty. Offsets in the range $-0.05$ to $-0.17$ show that stellar half-mass radii are generally smaller than the optical radius. The scatter is comparable to the formal uncertainties, which implies that the uncertainties are certainly not underestimated.
 (*fig:rm_ropt*)

</div>
<div id="div_fig2">

<img src="tmp_2307.03264/./m_r1.png" alt="Fig9.1" width="50%"/><img src="tmp_2307.03264/./m_r2.png" alt="Fig9.2" width="50%"/>

**Figure 9. -**  Size-stellar mass distributions for two redshift bins ($0.5<z<1$ at the top; $1<z<1.5$ at the bottom) and three different size proxies: rest-frame optical half-light radius $R_{\rm{opt}}$(_left_; projected stellar half-mass radius $R_{\rm{M_\star,2D}}$(_middle_); deprojected stellar half-mass radius $R_{\rm{M_\star,3D}}$(_bottom_). The solid lines indicate the 16th, 50th and 84th percentiles of the size distribution in 0.2 dex wide bins of $M_\star$(Table \ref{tab:medians}). The dotted lines in the middle and right-hand panels repeat, for reference, the solid lines in the left-hand panels. The color-coding is the star-formation rate relative to the star-forming sequence defined by [Leja, Speagle and Ting (2022)]().
 (*fig:m_r*)

</div>
<div id="div_fig3">

<img src="tmp_2307.03264/./ml_ml.png" alt="Fig2" width="100%"/>

**Figure 2. -**  Comparison of rest-frame $V$-band stellar mass-to-light ratio as inferred from Prospector (x-axis) and our HST color-based proxy (y-axis). Across a 1.5 dex range in $M/L$ the HST color-based $M/L$ estimates agree well with the ground truth as inferred from fits to the full SED. The color-coding is the star-formation rate relative to the star-forming sequence defined by [Leja, Speagle and Ting (2022)]().
 (*fig:ml_ml*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.03264"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

227  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
